In [2]:
import numpy as np 
import pandas as pd 

In [3]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

print(df_train.shape)
print(df_test.shape)

(300000, 25)
(200000, 24)


In [45]:
def my_describe(df):
    summary = pd.DataFrame(df.dtypes, columns=['Type'])
    summary = summary.reset_index()
    summary['Variable'] = summary['index']
    summary = summary[['Variable', 'Type']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Row'] = df.loc[0].values
    return summary


describ = my_describe(df_train)
describ

,Variable,Type,Missing,Uniques,First Row
0,id,int64,0,300000,0
1,bin_0,int64,0,2,0
2,bin_1,int64,0,2,0
3,bin_2,int64,0,2,0
4,bin_3,object,0,2,T
5,bin_4,object,0,2,Y
6,nom_0,object,0,3,Green
7,nom_1,object,0,6,Triangle
8,nom_2,object,0,6,Snake
9,nom_3,object,0,6,Finland


In [67]:
def my_TE_encoder(df, var_names, trg_name):
    #for objects with lots of unique categories
    from category_encoders import TargetEncoder
    TE_features = TargetEncoder(cols=var_names, drop_invariant=True, return_df=True, min_samples_leaf=2, smoothing=1.0)
    TE_features.fit(df[var_names], df[trg_name])
    return TE_features

In [70]:
TE_feature_names = list(describ['Variable'][(describ.Type =='object') & (describ.Uniques>10)])
TE_feature_modifier = my_TE_encoder(df_train, TE_feature_names,['target'])
TE_feature_train = TE_feature_modifier.transform(df_train[TE_feature_names])
TE_feature_test = TE_feature_modifier.transform(df_test[TE_feature_names])

In [72]:
 #for uniques not greater than 10
OHE_feature_names = list(describ['Variable'][describ.Uniques<=10])

In [75]:
pd.get_dummies(pd.concat([df_train, df_test], axis=0), columns=OHE_feature_names)

G:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,id,month,nom_5,nom_6,nom_7,nom_8,nom_9,ord_3,ord_4,ord_5,...,ord_2_Warm,day_1,day_2,day_3,day_4,day_5,day_6,day_7,target_0.0,target_1.0
0,0,2,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51,h,D,kr,...,0,0,1,0,0,0,0,0,1,0
1,1,8,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21,a,A,bF,...,0,0,0,0,0,0,0,1,1,0
2,2,2,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0,h,R,Jc,...,0,0,0,0,0,0,0,1,1,0
3,3,1,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71,i,D,kW,...,0,0,1,0,0,0,0,0,0,1
4,4,8,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7,a,R,qP,...,0,0,0,0,0,0,0,1,1,0
5,5,2,46cab09da,29a854620,ff5b35098,b7e6f8e6f,51e27c16d,j,E,PZ,...,0,0,1,0,0,0,0,0,1,0
6,6,4,be5592604,3393a0f78,c6587685d,06f5ae149,7e3d79a0d,g,P,wy,...,0,0,0,0,0,1,0,0,1,0
7,7,2,72f8028dc,55eed5058,2dd9daf45,98addc2c9,feb72ecc2,j,K,Ed,...,0,0,0,0,1,0,0,0,1,0
8,8,4,4604905e7,3e44d44eb,3f0057c9b,a2d110837,34a7273bf,e,V,qo,...,0,0,0,1,0,0,0,0,1,0
9,9,2,ad95dc0ee,8ed6221ae,4fbfe4a84,2c15d0173,0ece7a511,h,Q,CZ,...,0,0,0,1,0,0,0,0,1,0


In [ ]:
df[train_part:].drop('target', axis=1)

Transfom features:<br>LabelEncoder for true/false features
<br>OneHotEncoder for features with number of unique values <=30
<br>TargetEncoder for features with many unique values
<br>LabelEncoder for other features
